## Построение первой графовой НС с использованием PyG

In [9]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super().__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, data):
        # edge_index - это тензор [2, num_edges]
        # отличие от DGL - модули PyG принимают
        # на вход эти тензоры, а не объект графа 
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [11]:
from torch_geometric.datasets import Planetoid
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = Planetoid(root='./tmp', name='Cora')
G = dataset[0].to(device)
model = GCN(dataset.num_node_features, dataset.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(G)
    loss = F.nll_loss(out[G.train_mask], G.y[G.train_mask])
    loss.backward()
    optimizer.step()

In [12]:
model.eval()
pred = model(G).argmax(dim=1)
correct = (pred[G.test_mask] == G.y[G.test_mask]).sum()
acc = int(correct) / int(G.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8120
